## 1. Carga e Importación de Librerías

In [70]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.utils import class_weight

## 2. Carga y División de Datos

Cargamos los conjuntos de datos originales de entrenamiento (`salario.csv`) y de prueba (`test.csv`).

A diferencia del notebook anterior (donde `X_val` era una partición de `salario.csv`), en este notebook `X_train` será el dataset `salario.csv` completo y `X_test_raw` será el dataset `test.csv` (utilizado en Kaggle), sobre el cual generaremos las predicciones finales.

In [71]:
df_train = pd.read_csv('./salario.csv', na_values=['?'], skipinitialspace=True)
X_test_raw = pd.read_csv('./salario_test.csv', na_values=['?'], skipinitialspace=True)

In [72]:
mapa_salario = {'<=50K': 0, '>50K': 1}
df_train['salario'] = df_train['salario'].map(mapa_salario)

target_col = 'salario'
y_train = df_train[target_col]
X_train_raw = df_train.drop(target_col, axis=1)

Se almacena la columna \`ID\` de \`X_test_raw\` para su uso en el archivo final de predicciones y se elimina del dataset de prueba antes de aplicar el preprocesado.

In [73]:
submission_ids = X_test_raw['ID']
X_test_raw = X_test_raw.drop('ID', axis=1)
submission_ids = X_test_raw.index

## 3. Preprocesado V2: Imputación y Selección

Se replica la estrategia de preprocesado ganadora (Pipeline V2) identificada en el notebook anterior.

### 3.1. Imputación de Valores Nulos

Se imputan los valores nulos (`?`) en las columnas categóricas (`trabajo`, `trabajo.1`, `pais-origen`) tanto en `X_train` como en `X_test_raw`. Para evitar cualquier *data leakage*, la moda utilizada para rellenar ambos conjuntos se calcula *únicamente* a partir del conjunto de entrenamiento (`X_train`).

In [74]:
columnas_con_nulos = ['trabajo', 'trabajo.1', 'pais-origen']
imputers = {}
for col in columnas_con_nulos:
    moda = X_train_raw[col].mode()[0]
    imputers[col] = moda
    X_train_raw[col] = X_train_raw[col].fillna(moda)
    X_test_raw[col] = X_test_raw[col].fillna(moda)

### 3.2. Aplicación del Preprocessor V2

Se instancia el mismo objeto ColumnTransformer del Pipeline V2, que aplica:
* StandardScaler a las variables numéricas.
* OrdinalEncoder a las variables ordinales ('estudios') y binarias ('sexo').
* OneHotEncoder a las variables nominales restantes.

Se aplica .fit\_transform() en `X_train` y solo .transform() en `X_test`.

In [75]:
numeric_features = ['edad', 'ganancias_inversiones', 'perdidas_inversiones', 'horas-trabajo_semana']
ordinal_features = ['estudios']
binary_features = ['sexo']
nominal_features = ['trabajo', 'estado-civil', 'trabajo.1', 'posicion-familiar', 'etnia', 'pais-origen']
education_order = [
    'Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', 
    '10th', '11th', '12th', 'HS-grad', 'Some-college', 
    'Assoc-voc', 'Assoc-acdm', 'Bachelors', 'Masters', 
    'Prof-school', 'Doctorate'
]
numeric_transformer_v2 = StandardScaler()
ordinal_transformer = OrdinalEncoder(
    categories=[education_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
binary_transformer = OrdinalEncoder()
nominal_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor_v2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_v2, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('bin', binary_transformer, binary_features),
        ('nom', nominal_transformer, nominal_features)
    ],
    remainder='passthrough'
)

In [76]:
X_train_processed = preprocessor_v2.fit_transform(X_train_raw)
X_test_processed = preprocessor_v2.transform(X_test_raw)

### 3.3. Selección de Características (Consenso V2)

Finalmente, se aplica la máscara de selección de características "Consenso $\geq$ 2" (identificada en el Apartado 1) a ambos conjuntos, reduciendo la dimensionalidad a las 24 features más robustas.

In [77]:
k_best_selector = SelectKBest(score_func=f_classif, k=30)
k_best_selector.fit(X_train_processed, y_train)
features_elegidas_filter = k_best_selector.get_support()
estimator_rfe = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42, class_weight='balanced')
rfe_selector = RFE(estimator=estimator_rfe, n_features_to_select=30, step=1)
rfe_selector.fit(X_train_processed, y_train)
features_elegidas_wrapper = rfe_selector.support_
lasso_model = LogisticRegression(
    penalty='l1', solver='liblinear', C=0.1, 
    random_state=42, max_iter=1000, class_weight='balanced'
)
embedded_selector = SelectFromModel(
    lasso_model, max_features=30, threshold=-np.inf
) 
embedded_selector.fit(X_train_processed, y_train)
features_elegidas_embedded = embedded_selector.get_support()
features_consenso_final = (features_elegidas_filter & features_elegidas_wrapper) | \
                          (features_elegidas_filter & features_elegidas_embedded) | \
                          (features_elegidas_wrapper & features_elegidas_embedded)
num_features_final = np.sum(features_consenso_final)
print(f"Número de características seleccionadas por consenso: {num_features_final}")

Número de características seleccionadas por consenso: 24


In [78]:
X_train = X_train_processed[:, features_consenso_final]
X_test = X_test_processed[:, features_consenso_final]
print(f"X_train listo con shape: {X_train.shape}")
print(f"y_train listo con shape: {y_train.shape}")
print(f"X_test listo con shape: {X_test.shape}")

X_train listo con shape: (27998, 24)
y_train listo con shape: (27998,)
X_test listo con shape: (4563, 24)


## 4. Selección del Modelo Final

En el notebook `supervised.ipynb` se realizó una optimización exhaustiva (GridSearchCV) de 9 familias de modelos, evaluándolos en nuestro conjunto de validación local (`X_test`). En esa comparativa, **XGBoost** obtuvo el F1-Score (Clase 1) más alto, con **0.7230**.

Sin embargo, al enviar las predicciones de los mejores modelos (XGBoost, LGBM, Keras) a la competición de Kaggle (que utiliza un conjunto de prueba desconocido y diferente), el modelo de Red Neuronal demostró una capacidad de generalización superior, alcanzando un F1-Score público de 0.861.

Dado que el objetivo final del proyecto es maximizar el rendimiento en datos nuevos y desconocidos seleccionamos la Red Neuronal como el modelo definitivo para este notebook de producción, a pesar de que el que mejor F-1 Score tuvo en nuestras pruebas fue Xgboost

## 5. Entrenamiento del Modelo Keras

Se instancia y entrena el modelo Keras ganador. La arquitectura (`Input(24) -> Dense(64) -> Dense(32) -> Dense(1)`) y los hiperparámetros (Adam, `validation_split=0.1`, `epochs=50`) son idénticos a los definidos en la fase de experimentación.

Crucialmente, se aplica la estrategia de manejo de desbalanceo (`class_weight='balanced'`) y se entrena sobre la totalidad del conjunto `X_train` (22,398 filas) para preparar el modelo para la predicción final.

In [79]:
def create_keras_model(meta, dropout_rate=0.3):
    n_features_in = meta["n_features_in_"]
    model = Sequential([
        Input(shape=(n_features_in,)),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy'
    )
    return model

class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(y_train),y=y_train)
keras_class_weights = dict(enumerate(class_weights))
print(f"Pesos de clase para Keras: {keras_class_weights}")

final_keras = KerasClassifier(
    model=create_keras_model,
    dropout_rate=0.3,
    epochs=50,
    batch_size=64,
    fit__class_weight=keras_class_weights,
    verbose=1
)
final_keras.fit(X_train, y_train)

Pesos de clase para Keras: {0: np.float64(0.6575387505871301), 1: np.float64(2.086911150864639)}
Epoch 1/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5232
Epoch 2/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4146
Epoch 3/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4043
Epoch 4/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3982
Epoch 5/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3934
Epoch 6/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3914
Epoch 7/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3875
Epoch 8/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3881  
Epoch 9/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3851
Epoch 10/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3839
Epoch 11/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3814  
Epoch 12/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3804
Epoch 13/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3807
Epoch 14/50
438/438 ━━━━

,model,<function cre...001E342C04FE0>
,build_fn,None
,warm_start,False
,random_state,None
,optimizer,'rmsprop'
,loss,None
,metrics,None
,batch_size,64
,validation_batch_size,None
,verbose,1
,callbacks,None


## 6. Generación de Predicciones Finales

Una vez entrenado el modelo Keras sobre todos los datos de `X_train`, se utiliza para predecir las probabilidades en el `X_test` preprocesado.

Se aplica el **umbral de decisión óptimo (0.69)**, identificado en el notebook anterior, para convertir las probabilidades en las clases finales (0 o 1). Finalmente, se mapean las clases a sus etiquetas originales (`<=50K` o `>50K`) y se genera el archivo `best_model_predictions.csv` en el formato `ID,salario` requerido.

In [80]:
y_pred_proba_keras = final_keras.predict_proba(X_test)

predictions_keras = (y_pred_proba_keras[:, 1] > 0.69).astype(int)

mapa_inverso = {v: k for k, v in mapa_salario.items()}

labels_keras = pd.Series(predictions_keras).map(mapa_inverso)


submission_ids = X_test_raw['ID'] if 'ID' in X_test_raw else np.arange(1, len(X_test_raw) + 1)


submission_keras = pd.DataFrame({
    'ID': submission_ids,
    'salario': labels_keras 
})
submission_keras.to_csv('test_labels.csv', index=False)

print(submission_keras.head())

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
   ID salario
0   1   <=50K
1   2   <=50K
2   3   <=50K
3   4    >50K
4   5   <=50K
